In [1]:
# new models, altough I see that media constrain prob has a huge effect on this models.

In [1]:
import cobra
import glob
import os
import time
import pandas as pd
from cobra.flux_analysis import pfba

In [2]:
model = cobra.io.load_matlab_model('/home/roger/fastcormics/data/median/SampleModel_mix.mat') #71 a 74  o 161 a 164  o 251 254

No defined compartments in model ContextModel. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


In [3]:
sol = pfba(model)

In [4]:
def apply_constraints(model, constraints_dict):
    new_model = model.copy()
    for rid, (lb, ub) in constraints_dict.items():
        rxn = new_model.reactions.get_by_id(rid)
        rxn.lower_bound = lb
        rxn.upper_bound = ub
    return new_model

normal_conditions = {
    "O2t": (0.000558159, 0.000757101),
    "O2tm": (0.000314589, 0.000413189), 
    "ATPS4mi": (0.000242538, 0.000312538),
    "EX_h[e]": (0.00007578, 0.00008422)
}

mix_conditions = {
    "O2t": (0.000740940988062504, 0.000918608719909532),
    "O2tm": (0.000389772, 0.000584972),
    "ATPS4mi": (0.000324995, 0.000394495),
    "EX_h[e]": (0.000083, 0.000111)
}


In [5]:
media_constrains = {
    "EX_arg_L[e]": (-1, 1),
    "EX_ca2[e]": (-1, 1),
    "EX_cl[e]": (-1, 0),
    "EX_co2[e]": (-1, 1),
    "EX_fe3[e]": (-1, 1),
    "EX_fol[e]": (-1, 0),
    "EX_chol[e]": (-1, 0),
    "EX_glc[e]": (-1, 1),
    "EX_gln_L[e]": (-1, 1),
    "EX_gly[e]": (-1, 0),
    "EX_h[e]": (-1, 1),
    "EX_h2o[e]": (-1, 1),
    "EX_his_L[e]": (-1, 0),
    "EX_inost[e]": (-1, 1),
    "EX_k[e]": (-1, 1),
    "EX_Lcystin[e]": (-1, 1),
    "EX_leu_L[e]": (-1, 1),
    "EX_lys_L[e]": (-1, 1),
    "EX_na1[e]": (-1, 0),
    "EX_o2[e]": (-1, 0),
    "EX_oh1[e]": (-1, 1),
    "EX_phe_L[e]": (-1, 0),
    "EX_pi[e]": (-1, 1),
    "EX_pnto_R[e]": (-1, 1),
    "EX_pydx[e]": (-1, 0),
    "EX_ribflv[e]": (-1, 0),
    "EX_so4[e]": (-1, 1),
    "EX_thr_L[e]": (-1, 1),
    "EX_trp_L[e]": (-1, 1),
    "EX_tyr_L[e]": (-1, 1),
    "EX_val_L[e]": (-1, 1),
    "EX_ncam[e]": (-1, 1),
    "EX_ser_L[e]": (-1, 1),
    "EX_ile_L[e]": (-1, 1),
    "EX_met_L[e]": (-1, 1),
    "EX_thm[e]": (-1, 0)
}


def apply_media_constraints(model, media_dict):
    new_model = model.copy()
    for rxn_id, (lb, ub) in media_dict.items():
        try:
            rxn = new_model.reactions.get_by_id(rxn_id)
            rxn.lower_bound = lb
            rxn.upper_bound = ub
        except KeyError:
            pass  # skip missing reactions
    return new_model


# final apply constrains function
def apply_cons(model, constrains1, constrains2):
    print("First Biomass pFBA:", pfba(model).objective_value)
    nw_mdl = apply_constraints(model, constrains1)
    print("Seahorse constrains pFBA:", pfba(nw_mdl).objective_value)
    fl_mdl = apply_media_constraints(nw_mdl, constrains2)
    print("Media paper constrains pFBA:", pfba(fl_mdl).objective_value)
    return nw_mdl


In [6]:
pro = apply_cons(model, mix_conditions, media_constrains)

First Biomass pFBA: 32487.65389967203
Seahorse constrains pFBA: 52.70344401433368
Media paper constrains pFBA: 1.2322214910748601


In [16]:
# after you’ve solved your model (pro.objective = …; pro.optimize())
rxn_ids = ["EX_glc_D[e]", "EX_lac_L[e]", "EX_gln_L[e]"]
fluxes = {rxn: pro.reactions.get_by_id(rxn).flux for rxn in rxn_ids}

print(f"Glucose exchange flux  : {fluxes['EX_glc_D[e]']}")
print(f"Lactate exchange flux : {fluxes['EX_lac_L[e]']}")
print(f"Glutamine exchange flux : {fluxes['EX_gln_L[e]']}")



Glucose exchange flux  : -0.8450585042789298
Lactate exchange flux : 0.6097327900660987
Glutamine exchange flux : -0.6115645431773508


In [11]:
glc_flux = abs(pro.reactions.get_by_id("EX_glc_D[e]").flux)

In [8]:
# ATP production
def summarize_met_flux(model, subsystems, met_id):
    sol    = pfba(model)
    fluxes = sol.fluxes.to_dict()
    rxns = [
        rxn for rxn in model.reactions
        if rxn.subsystem in subsystems and abs(fluxes.get(rxn.id, 0.0)) > 1e-6
    ]
    met = model.metabolites.get_by_id(met_id)
    produced = 0.0
    consumed = 0.0
    for rxn in rxns:
        v     = fluxes.get(rxn.id, 0.0)
        stoich = rxn.metabolites.get(met, 0.0)
        if stoich == 0:
            continue
        delta = stoich * v
        if delta > 0:
            produced += delta
        else:
            consumed += -delta
    net = produced - consumed
    print(f"Subsystems: {subsystems}")
    print(f"Metabolite: {met_id}")
    print(f"  Produced: {produced:.6f} mmol/gDW/h")
    print(f"  Consumed: {consumed:.6f} mmol/gDW/h")
    print(f"  Net:      {net:.6f} mmol/gDW/h")

    return net

In [9]:
atp_gly = summarize_met_flux(pro,
                                 subsystems={'Glycolysis/gluconeogenesis'},
                                 met_id='atp[c]')

Subsystems: {'Glycolysis/gluconeogenesis'}
Metabolite: atp[c]
  Produced: 2.739027 mmol/gDW/h
  Consumed: 1.662276 mmol/gDW/h
  Net:      1.076751 mmol/gDW/h


In [12]:
atp_gly / glc_flux

1.2741734385230392

In [17]:
def react_subsystem(model, subsystem):
    sol    = pfba(model)
    fluxes = sol.fluxes.to_dict()
    for rxn in model.reactions:
        if rxn.subsystem == subsystem and abs(fluxes.get(rxn.id, 0)) > 1e-6:
            print(f"{rxn.id}: {rxn.reaction}  |  Flux: {fluxes[rxn.id]:.8f}")


# Examples
#find_react(final_cons, 'TKT1')
#find_met(final_cons, "glu_L[c]") 
react_subsystem(pro, 'Citric acid cycle')   # it seems that is active MDHm

CSm: accoa[m] + h2o[m] + oaa[m] --> cit[m] + coa[m] + h[m]  |  Flux: 0.45532479
FUMm: fum[m] + h2o[m] <=> mal_L[m]  |  Flux: -0.07322042
ICDHy: icit[c] + nadp[c] --> akg[c] + co2[c] + nadph[c]  |  Flux: 0.91066121
ICDHyrm: icit[m] + nadp[m] <=> akg[m] + co2[m] + nadph[m]  |  Flux: -0.45533642
MDHm: mal_L[m] + nad[m] <=> h[m] + nadh[m] + oaa[m]  |  Flux: 0.45532479
SUCD1m: fad[m] + succ[m] <=> fadh2[m] + fum[m]  |  Flux: -0.00047657
r0509: q10[m] + succ[m] --> fum[m] + q10h2[m]  |  Flux: 0.00047657
ACONT: cit[c] <=> icit[c]  |  Flux: 0.45532479
FUM: fum[c] + h2o[c] <=> mal_L[c]  |  Flux: 0.07393680
MDH: mal_L[c] + nad[c] <=> h[c] + nadh[c] + oaa[c]  |  Flux: -0.45460841


In [18]:
react_subsystem(pro, 'Oxidative phosphorylation')

PPAm: h2o[m] + ppi[m] --> h[m] + 2.0 pi[m]  |  Flux: 0.30421729
ATPS4mi: adp[m] + 4.0 h[i] + pi[m] --> atp[m] + h2o[m] + 3.0 h[m]  |  Flux: 0.00039450
CYOR_u10mi: 2.0 ficytC[m] + 2.0 h[m] + q10h2[m] --> 2.0 focytC[m] + 4.0 h[i] + q10[m]  |  Flux: 0.00077954
CYOOm3i: 4.0 focytC[m] + 7.92 h[m] + o2[m] --> 4.0 ficytC[m] + 1.96 h2o[m] + 4.0 h[i] + 0.02 o2s[m]  |  Flux: 0.00038977


In [20]:
def find_met(model, met_id):
    sol = pfba(model)
    fluxes = sol.fluxes.to_dict()
    met = model.metabolites.get_by_id(met_id)
    active_reactions = [
        rxn for rxn in met.reactions
        if abs(fluxes.get(rxn.id, 0.0)) > 1e-6
    ]
    for rxn in active_reactions:
        print(f"{rxn.id}: {rxn.reaction}  |  Flux: {fluxes[rxn.id]:.6f}")

In [22]:
find_met(pro, 'gln_L[e]')  #el glutamat enters a loop, is not being used for TCA (mix model) , 'glu_L[m]',  it seems than in normal neither.

r1579: gln_L[e] + pro_L[c] <=> gln_L[c] + pro_L[e]  |  Flux: 0.611565
EX_gln_L[e]: gln_L[e] <=>   |  Flux: -0.611565


In [23]:
find_met(pro, 'gln_L[c]')

GLNtm: gln_L[c] --> gln_L[m]  |  Flux: 0.609605
GLUPRT: gln_L[c] + h2o[c] + prpp[c] --> glu_L[c] + ppi[c] + pram[c]  |  Flux: 0.000294
CBPS: 2.0 atp[c] + gln_L[c] + h2o[c] + hco3[c] --> 2.0 adp[c] + cbp[c] + glu_L[c] + 2.0 h[c] + pi[c]  |  Flux: 0.000303
r1579: gln_L[e] + pro_L[c] <=> gln_L[c] + pro_L[e]  |  Flux: 0.611565
PRFGS: atp[c] + fgam[c] + gln_L[c] + h2o[c] --> adp[c] + fpram[c] + glu_L[c] + h[c] + pi[c]  |  Flux: 0.000294
biomass_maintenance: 0.50563 ala_L[c] + 0.35926 arg_L[c] + 0.27942 asn_L[c] + 0.35261 asp_L[c] + 20.7045 atp[c] + 0.020401 chsterol[c] + 0.011658 clpn_hs[c] + 0.039036 ctp[c] + 0.046571 cys_L[c] + 0.27519 g6p[c] + 0.326 gln_L[c] + 0.38587 glu_L[c] + 0.53889 gly[c] + 0.036117 gtp[c] + 20.6508 h2o[c] + 0.12641 his_L[c] + 0.28608 ile_L[c] + 0.54554 leu_L[c] + 0.59211 lys_L[c] + 0.15302 met_L[c] + 0.023315 pail_hs[c] + 0.15446 pchol_hs[c] + 0.055374 pe_hs[c] + 0.002914 pglyc_hs[c] + 0.25947 phe_L[c] + 0.41248 pro_L[c] + 0.005829 ps_hs[c] + 0.39253 ser_L[c] + 0.0

In [24]:
find_met(pro, 'gln_L[m]') #goes glutamine into mitochondria and transforms into glutamate

GLNtm: gln_L[c] --> gln_L[m]  |  Flux: 0.609605
GLUNm: gln_L[m] + h2o[m] --> glu_L[m] + nh4[m]  |  Flux: 0.609605


In [25]:
find_met(pro, 'glu_L[m]') #transforms into glu5sa

FPGS8m: 10fthf5glu[m] + atp[m] + glu_L[m] --> 10fthf6glu[m] + adp[m] + h[m] + pi[m]  |  Flux: 0.000522
GLUDxm: glu_L[m] + h2o[m] + nad[m] <=> akg[m] + h[m] + nadh[m] + nh4[m]  |  Flux: -0.609605
r0074: glu5sa[m] + h2o[m] + nad[m] <=> glu_L[m] + 2.0 h[m] + nadh[m]  |  Flux: -1.066095
GLUNm: gln_L[m] + h2o[m] --> glu_L[m] + nh4[m]  |  Flux: 0.609605
FPGS7m: 10fthf[m] + atp[m] + 4.0 glu_L[m] --> 10fthf5glu[m] + adp[m] + 3.0 h2o[m] + h[m] + pi[m]  |  Flux: 0.000522
FPGS9m: 10fthf6glu[m] + atp[m] + glu_L[m] --> 10fthf7glu[m] + adp[m] + h[m] + pi[m]  |  Flux: 0.000522
GLUt2m: glu_L[c] + h[c] <=> glu_L[m] + h[m]  |  Flux: -0.149985


In [27]:
find_met(pro, 'glu5sa[c]') #transport into glu5sa[c]

G5SADs: glu5sa[c] --> 1pyr5c[c] + h2o[c] + h[c]  |  Flux: 1.066095
GLU5SAtmc: glu5sa[m] <=> glu5sa[c]  |  Flux: 1.066095


In [28]:
find_met(pro, '1pyr5c[c]') #transforms into pro_L[c]?

G5SADs: glu5sa[c] --> 1pyr5c[c] + h2o[c] + h[c]  |  Flux: 1.066095
P5CR: 1pyr5c[c] + 2.0 h[c] + nadph[c] --> nadp[c] + pro_L[c]  |  Flux: 1.066095


In [29]:
find_met(pro, 'pro_L[c]') #i guess it is co-transporter of aa into cell, basically an antiporter.   

r1612: pro_L[c] + trp_L[e] <=> pro_L[e] + trp_L[c]  |  Flux: 0.000044
r1566: ala_L[e] + pro_L[c] <=> ala_L[c] + pro_L[e]  |  Flux: 0.001656
r1648: asn_L[c] + pro_L[e] <=> asn_L[e] + pro_L[c]  |  Flux: -0.000915
r1602: met_L[e] + pro_L[c] <=> met_L[c] + pro_L[e]  |  Flux: 0.150977
r1579: gln_L[e] + pro_L[c] <=> gln_L[c] + pro_L[e]  |  Flux: 0.611565
r1591: pro_L[c] + ser_L[e] <=> pro_L[e] + ser_L[c]  |  Flux: 0.293309
P5CR: 1pyr5c[c] + 2.0 h[c] + nadph[c] --> nadp[c] + pro_L[c]  |  Flux: 1.066095
r1649: pro_L[e] + val_L[c] <=> pro_L[c] + val_L[e]  |  Flux: -0.001155
r1642: leu_L[e] + pro_L[c] <=> leu_L[c] + pro_L[e]  |  Flux: 0.001787
r1621: phe_L[e] + pro_L[c] <=> phe_L[c] + pro_L[e]  |  Flux: 0.000850
r1650: pro_L[e] + thr_L[c] <=> pro_L[c] + thr_L[e]  |  Flux: -0.001024
r1629: pro_L[c] + tyr_L[e] <=> pro_L[e] + tyr_L[c]  |  Flux: 0.000523
biomass_maintenance: 0.50563 ala_L[c] + 0.35926 arg_L[c] + 0.27942 asn_L[c] + 0.35261 asp_L[c] + 20.7045 atp[c] + 0.020401 chsterol[c] + 0.011658 c